## Python's Heapq module (for min/max heaps)

In [1]:
import heapq
import math
'''
heapify(iterable) : convert iterable into heap
heappush(heap, element) : insert element into heap and restructure if necessary
heappop(heap) : remove smallest element from the heap and restructure the heap
heappushpop(heap, ele) : combines push and pop  (first push, then pop smallest)
heapreplace(heap, ele) : (first pop smallest, then push)
nlargest(k, iterable, key = function) : return k largest elements from iterable 
nsmallest(k, iterable, key = function) : return k smallest elements from iterable
'''

l1 = [4,5,2,3,9,0,10]
heapq.heapify(l1)
print(l1)
heapq.heappush(l1, -1)
print(l1)

[0, 3, 2, 5, 9, 4, 10]
[-1, 0, 2, 3, 9, 4, 10, 5]


## Stack where Min and Max elements are obtained in O(1)
##### using another stack

In [1]:
class custom_stack():
    def __init__(self):
        self.list = []
        self.aux_max = []
        self.aux_min = []
        
    def push(self, e):
        self.list.append(e)
        if not len(self.aux_max):
            self.aux_max.append(e)
        else: 
            if self.aux_max[-1] < e:
                self.aux_max.append(e)
        if not len(self.aux_min):
            self.aux_min.append(e)
        else: 
            if self.aux_min[-1] > e:
                self.aux_min.append(e)
    def get_min(self):
        return self.aux_min[-1]
    
    def get_max(self):
        return self.aux_max[-1]
    
st = custom_stack()
st.push(1)
st.push(4)
st.push(2)
st.push(-1)

print(st.get_min(), st.get_max(), sep = '\t')

-1	4


## Stack with minimum in o(1)
##### without using another stack

In [10]:
class CustomStack():
    
    def __init__(self, func_dict):
        self.stack = []
        self.fdict = func_dict
        
    def push(self, val):
        val_dict = {}
        val_dict['val'] = val
        if self.is_empty():
            for i in self.fdict:
                val_dict[i] = val
        else:
            for i in self.fdict:
                val_dict[i] = self.fdict[i](val, self.stack[-1][i])
        self.stack.append(val_dict)
            
    
    def pop(self):
        return self.stack.pop()['val']
    
    def get_f(self, func):
        return self.stack[-1][func]
    
    def is_empty(self):
        return not len(self.stack)
    
    def __repr__(self):
        s = ''
        for i in self.stack:
            s = str(i['val']) + s
            s = '\n'+s     
        s += '\n\n'
        for i in self.fdict:
            s += (i + ' : '+(str(self.stack[-1][i]) + ',  '))
        return s
    

In [16]:
s = CustomStack({'min':(lambda x, y : x if x < y else y), 'max':(lambda x, y : x if x > y else y)})
s.push(1)
s.push(-1)
s.push(6)
s.push(3)
#s.is_empty()
s


3
6
-1
1

min : -1,  max : 6,  

## Queue with minimum in o(1) (using 2 stacks)

In [12]:
class CustomQueue():
    
    def __init__(self, fdict):
        self.s1 = CustomStack(fdict)         #for adding(rear)
        self.s2 = CustomStack(fdict)         #for removing(front)
        self.fdict = fdict
        
    def add(self, val):
        self.s1.push(val)
    
    def remove(self):
        if self.s2.is_empty():
            while not self.s1.is_empty():
                self.s2.push(self.s1.pop())
        return self.s2.pop()
    
    def get_f(self, func):
        if self.s1.is_empty() or self.s2.is_empty():
            return self.s1.get_f(func) if self.s2.is_empty() else self.s2.get_f(func)
        return self.fdict[func](self.s1.get_f(func), self.s2.get_f(func))
    
    def is_empty(self):
        return self.s1.is_empty() and self.s2.is_empty()

In [19]:
cq = CustomQueue({'min':(lambda x, y : x if x < y else y), 'max':(lambda x, y : x if x > y else y)})
cq.add(1)
cq.add(2)
cq.add(6)
#cq.add(-1)
cq.remove()

cq.get_f('min')

-1

## Stack using singly linked lists

##### overcomes the issue of amortized o(1) time of pushing and popping of an array based stack, which is caused due to resizing

In [13]:
class Empty(Exception):
    """ To raise empty container exceptions"""
    pass

class linked_stack():
    class _node(): #nested class for a node
        __slots__ = '_data', '_next' #fixed amount of memory usage for each node
        
        def __init__(self, data, next):
            self._data = data
            self._next = next
    
    def __init__(self):
        self._head = None
        self._size = 0
        
    def __len__(self):
        return self._size
    
    def is_empty(self):
        return self._size == 0 
    
    def push(self, data):
        self._head = self._node(data, self._head)
        self._size += 1
    
    def top(self):
        if self.is_empty():
            raise Empty('Stack is empty')
        return self._head._data
    
    def pop(self):
        if self.is_empty():
            raise Empty('Stack is empty, cannot pop element')
        top = self._head._data
        self._head = self._head._next
        self._size -= 1
        return top
    
    def display(self):
        if self.is_empty():
            raise Empty('No elements to display')
        element = self._head
        while element:
            print(element._data)
            element = element._next
        

## Queue using SLLs:
##### to avoid amortized o(1) for push_front() and pop_rear()

In [14]:
class linked_queue():
    
    class _node():
        __slots__ = '_data', '_next' #fixed amount of memory usage for each node
        
        def __init__(self, data, next):
            self._data = data
            self._next = next
    
    def __init__(self):
        ''' Create an empty queue '''
        self._head = None
        self._tail = None
        self._size = 0
        
    def __len__(self):
        return self._size
    
    def is_empty(self):
        return self._size == 0

    def front(self):
        if self.is_empty():
            raise Empty('Queue is empty')
        return self._head._data
    
    def enq(self, data):
        new_node = self._node(data, None)
        if self.is_empty():
            self._head = new_node
        else:
            self._tail._next = new_node
        self._tail = new_node
        self._size += 1
    
    def deq(self):
        if self.is_empty():
            raise Empty('Queue is empty')
        front_element = self._head._data
        self._head = self._head._next
        self._size -= 1
        if self.is_empty():
            self._tail = None
        return front_element
    
    def display(self):
        if self.is_empty():
            raise Empty('No elements to display')
        element = self._head
        while element:
            print(element._data)
            element = element._next

## Circular queue
##### especially for round robin scheduling

In [1]:
class circular_queue():
    class _node:
            __slots__ = '_data', '_next' #fixed amount of memory usage for each node
        
            def __init__(self, data, next):
                self._data = data
                self._next = next
            
    def __init__(self):
        self._tail = None        #only 1 reference for a circular q 
        self._size = 0
        
    def __len__(self):
        return self._size
    
    def is_empty(self):
        return self._size == 0

    def front(self):
        if self.is_empty():
            raise Empty('Queue is empty')
        return self._tail._next._data
    
    def enq(self, data):
        new_element = self._node(data, None)
        if self.is_empty():
            new_element._next = new_element
        else:
            new_element._next = self._tail._next
            self._tail._next = new_element
        self._tail = new_element
        self._size += 1
        
    def deq(self):
        if self.is_empty():
            raise Empty('Queue is empty')
        oldhead = self._tail._next
        if self._size == 1:
            self._tail = None
        else:
            self._tail._next = oldhead._next
        self._size -= 1
        return oldhead._data
            
    
    def display(self):
        ptr = self._tail._next
        while ptr is not self._tail:
            print(ptr._data)
            ptr = ptr._next
        print(ptr._data)

## Sparse tables
##### for O(logn) ranged queries, O(1) for range minimum/maximum queries (for idempotent functions, o(1))

In [84]:
''' 
every positive number can be represented as sum of powers of 2 (8-4-2-1 method).
and so can be a range.
for example -> f(arr[2,13]) can be represented as: (12 elements)
f(arr[2,9]) U f(arr[10,13]), resulting in O(logn) time

for overlapping/idempotent functions, such as minimum, maximum, we can have overlapping ranges, such as
for example -> f(arr[2,13]) can be represented as: (12 elements)
f(arr[2,9]) U f(arr[6,13]), resulting in O(1) time (considering only the highest power of 2 less than r)
'''
import math

class SparseTable():
    def __init__(self, arr):
        self.arr = arr
        self.max_size = len(arr)
        self._build()
        
    def _build(self):                    #O(logn)
        col_size = int(math.log2(self.max_size)) + 2
        self.matrix = [[None] * col_size for i in range(self.max_size)]
        for i in range(self.max_size):
            self.matrix[i][0] = self.arr[i]
        self.col_size = col_size   
        for j in range(1, col_size+1):
            i = 0
            while i + (1 << j) <= self.max_size:              #matrix[i][j] = sum(arr[i]...arr[i + 2 ** j])
                self.matrix[i][j] = self.matrix[i][j-1] + self.matrix[i + (1<<(j-1))][j-1]
                i += 1
    
    def query(self, l, r):
        sum = 0
        while l <= r:
            maxp2 = int(math.log2(r-l+1))
            sum += self.matrix[l][maxp2]          #matrix[i][j] = sum(arr[i]...arr[i + 2 ** j])
            l += (1 << maxp2)
        return sum

In [85]:
s = SparseTable(list(range(1,int(1e5))))
s.query(0,9)

55

## Sparse Table (for idempotent functions such as minimum/maximum)

In [88]:
import math

class SparseIdempotent():     #minimum
    def __init__(self, arr):
        self.arr = arr
        self.max_size = len(arr)
        self._build()
        
    def _build(self):                    #O(logn)
        col_size = int(math.log2(self.max_size)) + 2
        self.matrix = [[None] * col_size for i in range(self.max_size)]
        for i in range(self.max_size):
            self.matrix[i][0] = self.arr[i]
        self.col_size = col_size   
        for j in range(1, col_size+1):
            i = 0
            while i + (1 << j) <= self.max_size:              #matrix[i][j] = min(arr[i]...arr[i + 2 ** j])
                self.matrix[i][j] = min(self.matrix[i][j-1], self.matrix[i + (1<<(j-1))][j-1])
                i += 1
    
    def query(self, l, r):
        maxp2 = int(math.log2(r-l+1))
        return min(self.matrix[l][maxp2], self.matrix[r - (1 << maxp2) + 1][maxp2])

In [99]:
s = SparseIdempotent([1,2,3,4,5,6])
s.query(1,4)

2

## Disjoint Sparse Table : 
##### For o(1) querying of both associative and idempotent functions

#### https://discuss.codechef.com/t/tutorial-disjoint-sparse-table/17404